In [1]:
import pandas as pd
import numpy as np

In [2]:
data_import = pd.read_csv('data.csv')

In [3]:
print(data_import.shape)
data_import.describe()

(309, 10)


,name,diet,period,lived_in,type,length,taxonomy,named_by,species,link
count,309,309,309,308,309,291,309,309,304,309
unique,309,5,154,32,7,73,102,277,272,309
top,aardonyx,herbivorous,Late Cretaceous 76-74 million years ago,USA,sauropod,6.0m,Dinosauria Ornithischia Genasauria Cerapoda Or...,Marsh (1877),mongoliensis,https://www.nhm.ac.uk/discover/dino-directory/...
freq,1,185,14,77,69,21,11,4,7,1


In [4]:
data_import.set_index('name', inplace=True)

In [5]:
data_import.type.nunique()
data_import.type.value_counts()

sauropod             69
large theropod       60
small theropod       60
euornithopod         59
armoured dinosaur    32
ceratopsian          28
1.0m                  1
Name: type, dtype: int64

### We should modify this 1.0m since it is probably the lenghts of a dinosaur rather than the type

In [6]:
data_import.loc[data_import.type=='1.0m', 'length'] = '1.0m'
data_import.loc[data_import.type=='1.0m', 'type'] = np.nan


In [7]:
data_import.type.nunique()
data_import.type.value_counts()

sauropod             69
large theropod       60
small theropod       60
euornithopod         59
armoured dinosaur    32
ceratopsian          28
Name: type, dtype: int64

In [8]:
data_import.type.isna().sum() #Apparently it worked

1

In [9]:
data_import.loc[data_import.type.isna()] #Yes it worked, we change it manually, great!

,diet,period,lived_in,type,length,taxonomy,named_by,species,link
name,,,,,,,,,
lesothosaurus,herbivorous,Early Jurassic 213-200 million years ago,Lesotho,NaN,1.0m,Dinosauria Ornithischia,Galton (1978),diagnosticus,https://www.nhm.ac.uk/discover/dino-directory/...


# Parse columns

## Now we should parse columns period, lenght and named by

In [10]:
split_period = data_import["period"].str.split(" ", expand=True)
split_period[["Columna1", "Columna2", "Columna3", "Coulumna4", "Columna5", "Columna6"]] = data_import["period"].str.split(" ", expand=True)


In [11]:
split_period["Period_name"] = split_period["Columna1"] + " " + split_period["Columna2"] #Join column 0 and 1 to get the name of the period

In [12]:
split_period[["Start", "End"]] = split_period["Columna3"].str.split("-", expand=True)

In [13]:
split_period_clean = split_period.loc[:, ["Period_name", "Start", "End"]]

In [14]:
split_period_clean.head()

,Period_name,Start,End
name,,,
aardonyx,Early Jurassic,199,189
abelisaurus,Late Cretaceous,74,70
achelousaurus,Late Cretaceous,83,70
achillobator,Late Cretaceous,99,84
acrocanthosaurus,Early Cretaceous,115,105


## Merge it into the base data

In [15]:
data_import.drop(columns="period", inplace=True) # Drop the period column 

In [16]:
base_data = pd.merge(data_import, split_period_clean, left_index=True, right_index=True) #Here we will merge all the others df such as named by split and lenght

## Parse named by column into named_by and year

In [17]:
data_import[["Named_by", "Year"]] = data_import['named_by'].str.split("(", expand=True) #Splitting column by (

In [18]:
split_named = data_import.loc[:, ["Named_by", "Year"]] #Creating df for the splitted items

In [19]:
split_named['Year']=split_named["Year"].str.rstrip(')') #Remove the ) on last position

In [20]:
base_data = pd.merge(base_data, split_named, left_index=True, right_index=True)

## Parse lenght into lenght and units of measure

In [21]:
def parse_length(s):
    # parse field "length" into "length" ,"unit of measure"
    if pd.isnull(s):
        return (np.nan, np.nan)
    return (float(str(s)[:-1]), str(s)[-1:])

In [22]:
data_import['length_parsed'],data_import['measure'] = zip(*data_import['length'].apply(parse_length))
data_import[['length','length_parsed', 'measure']].sample(3)

,length,length_parsed,measure
name,,,
aralosaurus,8.0m,8.0,m
ornithomimus,4.0m,4.0,m
minmi,3.0m,3.0,m


In [23]:
parsed_length = data_import[['length_parsed', 'measure']]

In [24]:
base_data = pd.merge(base_data, parsed_length, left_index=True, right_index=True)

In [25]:
list(base_data.columns)

['diet',
 'lived_in',
 'type',
 'length',
 'taxonomy',
 'named_by',
 'species',
 'link',
 'Period_name',
 'Start',
 'End',
 'Named_by',
 'Year',
 'length_parsed',
 'measure']

In [26]:
base_data.drop(columns=['length', 'named_by'], inplace=True)

In [27]:
base_data.measure.nunique() #Only one unit of measure

1

In [28]:
base_data.loc[base_data['species'].isnull()][['Named_by', 'Year','species']] # These are not names but specie

,Named_by,Year,species
name,,,
centrosaurus,apertus,None,NaN
chasmosaurus,belli,None,NaN
coelophysis,bauri,None,NaN
dilophosaurus,wetherilli,None,NaN
euoplocephalus,tutus,None,NaN


### Modifying the values from Named_by to species

In [29]:
base_data['species'] = np.where(base_data['species'].isnull(), base_data['Named_by'], base_data['species'])
base_data.loc[base_data['Year'].isnull(),'Named_by'] = np.nan
base_data.loc[base_data['Year'].isnull()][['Named_by', 'Year','species']] 

,Named_by,Year,species
name,,,
albertaceratops,NaN,None,nesmoi
centrosaurus,NaN,None,apertus
chasmosaurus,NaN,None,belli
coelophysis,NaN,None,bauri
dilophosaurus,NaN,None,wetherilli
euoplocephalus,NaN,None,tutus


In [30]:
base_data.drop(columns=['named by', 'Named by'], inplace=True) #I run it twice so now can't find the columns

KeyError: "['named by' 'Named by'] not found in axis"

In [31]:
base_data.drop(columns=['link'],inplace=True)

In [32]:
list(base_data.columns)

['diet',
 'lived_in',
 'type',
 'taxonomy',
 'species',
 'Period_name',
 'Start',
 'End',
 'Named_by',
 'Year',
 'length_parsed',
 'measure']

In [33]:
base_data.rename(columns = {'diet':'Diet', 'lived_in': 'Location', 'type': 'Type', 'taxonomy':'Taxonomy', 'species':'Species', 'Period_name':'Period', 'length_parsed': 'length', 'measure':'Units of measure'}, inplace=True)

In [36]:
base_data.to_excel('./dinosaur_clean_data.xlsx')